In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import tensorflow as tf
import csv, glob
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint



Using TensorFlow backend.


In [2]:
ECG5 = np.load('C:/Users/Admini/Desktop/ECG 관련 데이터/윤지_ECG_array.npy')

In [3]:
ECG5 = np.asarray(ECG5)

In [4]:
ECG5.shape

#즉, 5000개 데이터가 12*5000이 있다.

(5000, 12, 5000)

In [5]:
ECG5

array([[[-253.76, -253.76, -253.76, ...,  -34.16,   24.4 ,    4.88],
        [ 263.52,  263.52,  263.52, ...,  -68.32,  -48.8 ,  -34.16],
        [ 517.28,  517.28,  517.28, ...,  -34.16,  -73.2 ,  -39.04],
        ...,
        [ 810.08,  810.08,  810.08, ..., -204.96, -200.08, -170.8 ],
        [ 810.08,  810.08,  810.08, ..., -200.08, -195.2 , -165.92],
        [ 527.04,  527.04,  527.04, ...,  102.48,   92.72,  112.24]],

       [[  -9.76,  -24.4 ,  -19.52, ...,   14.64,    9.76,    9.76],
        [   9.76,  -19.52,  -19.52, ...,    9.76,    9.76,    9.76],
        [  19.52,    4.88,    0.  , ...,   -4.88,    0.  ,    0.  ],
        ...,
        [  53.68,   34.16,   34.16, ...,  -14.64,  -19.52,  -24.4 ],
        [  48.8 ,   34.16,   34.16, ...,  -19.52,  -24.4 ,  -29.28],
        [   0.  ,  -14.64,   -9.76, ...,    4.88,    0.  ,   -4.88]],

       [[ -29.28,  -19.52,   -9.76, ...,   73.2 ,   78.08,   73.2 ],
        [ -63.44,  -53.68,  -43.92, ...,  -68.32,  -68.32,  -73.2 ],
    

In [6]:
diag = pd.read_excel("Diagnostics.xlsx")
diag5 = diag[:5000]

label_df=diag5[['PatientAge','Gender']]


label_df.loc[label_df.Gender=='MALE','Gender']=1
label_df.loc[label_df.Gender=='FEMALE','Gender']=0


#diagnostics데이터를 불러워서, 5000개를 수집한다.
# 환자나이와, 성별을 따로 불러워서 label_df에 올린다.
#이걸, 0,1 성별을 나눠서 열을 하나 추가한다.



C:\Users\Admini\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [7]:
label_df
Age=label_df.PatientAge
Sex=label_df.Gender

print(Sex.head(5))

#만든 label_df의 0,1값들을 를 age, sex로 이름을 수정한다.


0    1
1    0
2    0
3    1
4    0
Name: Gender, dtype: object


In [8]:
# TRAIN, VALIDATION, TEST로 데이터 나누기.


X_train_pre, X_test_external, Y_train_pre, Y_test_external = train_test_split(ECG5, Sex, train_size=0.64, test_size=0.36, random_state=42)

X_train_real, X_test_internal, Y_train_real, Y_test_internal = train_test_split(X_train_pre, Y_train_pre, train_size=0.75, test_size=0.25, random_state=42)


In [9]:
print(X_train_pre.shape)

print(X_test_external.shape)

print(Y_train_pre.shape)

print(Y_test_external.shape)


print(X_train_real.shape)

print(X_test_internal.shape)


print(Y_train_real.shape)

print(Y_test_internal.shape)

(3200, 12, 5000)
(1800, 12, 5000)
(3200,)
(1800,)
(2400, 12, 5000)
(800, 12, 5000)
(2400,)
(800,)


In [10]:
#2D -> 3D 로 차원 바꾸기


X_train_pre_3d = X_train_pre.reshape(3200,12,5000,1)

X_test_external_3d = X_test_external.reshape(1800,12,5000,1)

X_train_real_3d = X_train_real.reshape(2400,12,5000,1)

X_test_internal_3d = X_test_internal.reshape(800,12,5000,1)

In [11]:
#y값을 ONE HOT ENCODING 해야함.

Y_train_one=to_categorical(Y_train_real,2)

Y_val_one=to_categorical(Y_test_internal,2)

Y_test_one=to_categorical(Y_test_external,2)




In [15]:
#모델 만들기.
# TEMPERAL


# Temporal Analysis

model=Sequential()

model.add(ZeroPadding2D(padding=(0,2560),
                       input_shape=(12,5000,1)))

model.add(Conv2D(filters=16,
                kernel_size=(1,7)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((1,2)))

model.add(Conv2D(filters=16,
                kernel_size=(1,5)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((1,4)))

model.add(Conv2D(filters=32,
                kernel_size=(1,5)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((1,2)))

model.add(Conv2D(filters=32,
                kernel_size=(1,5)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((1,4)))

model.add(Conv2D(filters=64,
                kernel_size=(1,5)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((1,2)))

model.add(Conv2D(filters=64,
                kernel_size=(1,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((1,2)))

model.add(Conv2D(filters=64,
                kernel_size=(1,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((1,2)))

model.add(Conv2D(filters=64,
                kernel_size=(1,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

In [16]:
# Spatial 모델

model.add(Conv2D(filters=128, kernel_size=(12,1),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(1,2))

model.add(Flatten())
#  이미지 형태의 데이터를 배열 형태로 만드는 것.


In [17]:
# fully

model.add(Dense(128,input_dim=(1,5080)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3)) 


model.add(Dense(64,input_dim=(1,5080)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3)) 

model.add(Dense(2,activation='softmax'))


In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_2 (ZeroPaddin (None, 12, 10120, 1)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 12, 10114, 16)     128       
_________________________________________________________________
batch_normalization_8 (Batch (None, 12, 10114, 16)     64        
_________________________________________________________________
activation_1 (Activation)    (None, 12, 10114, 16)     0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 5057, 16)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 5053, 16)      1296      
_________________________________________________________________
batch_normalization_9 (Batch (None, 12, 5053, 16)     

In [19]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_real_3d, Y_train_one,
                   validation_data=(X_test_internal_3d,Y_val_one),
                    
                    batch_size=32,
                    
                    nb_epoch=500, verbose=2,
                    
                    callbacks=[EarlyStopping(patience=20),ModelCheckpoint(filepath='best-model.h5', save_best_only=True)]
                   )